In [25]:
import cv2
import random
import numpy as np
import copy
random.seed(72)

In [100]:
photos = cv2.imread('2-image.jpg')
print(photos.shape)

(360, 640, 3)


In [101]:
initial_clusters_pos = []
for i in range(3):
    x, y = random.randrange(photos.shape[0]), random.randrange(photos.shape[1])
    ci = photos[x,y,:]
    initial_clusters_pos.append([x,y])
print(initial_clusters_pos)

[[228, 70], [151, 93], [258, 325]]


In [102]:
def cal_k_mean(clusters_rgb, clusters_pos):
    #print(clusters_rgb)
    clusters_pts = [[], [], []]
#     for i in range(len(clusters_pos)):
#         clusters_pts.append([clusters_pos[i]])
    for i in range(photos.shape[0]):
        for j in range(photos.shape[1]):
            c = 0
            distance = []
            distance.append(np.linalg.norm(photos[i,j] - clusters_rgb[0], ord=2))
            distance.append(np.linalg.norm(photos[i,j] - clusters_rgb[1], ord=2))
            distance.append(np.linalg.norm(photos[i,j] - clusters_rgb[2], ord=2))

            c = np.argmin(distance)
            clusters_pts[c].append([i,j])
    return clusters_pts

In [103]:
def cal_mean(clusters_pts):
    new_clusters = []
    clusters_rgb = []
    for i in range(len(clusters_pts)):
        #print(len(clusters_pts[i]))
        center_x, center_y = np.mean(clusters_pts[i], axis=0)
        new_clusters.append([center_x, center_y])
        
        r_all, g_all, b_all = 0, 0, 0
        
        for j in range(len(clusters_pts[i])):
#             print(clusters_pts[i][j])
            r_all+= photos[clusters_pts[i][j][0], clusters_pts[i][j][1]][0]
            g_all+= photos[clusters_pts[i][j][0], clusters_pts[i][j][1]][1]
            b_all+= photos[clusters_pts[i][j][0], clusters_pts[i][j][1]][2]
        
        center_r = r_all / len(clusters_pts[i])
        center_g = g_all / len(clusters_pts[i])
        center_b = b_all / len(clusters_pts[i])
            
        clusters_rgb.append([center_r, center_g, center_b])
    
    return new_clusters, clusters_rgb

In [104]:
def k_means():
    repeat = True

    # mean 顏色
    clusters = []
    for pos in initial_clusters_pos:
        clusters.append(photos[pos[0], pos[1]])
    print(clusters)

    # 同一群pts
    clusters_pts = cal_k_mean(clusters, initial_clusters_pos)

    # 新的mean 位置和顏色
    new_clusters_pos, clusters_rgb = cal_mean(clusters_pts)

    while(repeat):
        repeat = False
        ori = copy.deepcopy(new_clusters_pos)
        clusters_pts = cal_k_mean(clusters_rgb, new_clusters_pos)
        new_clusters_pos, clusters_rgb = cal_mean(clusters_pts)
        for j in range(len(new_clusters_pos)):
            if ((new_clusters_pos[j][0] - ori[j][0])>1e-6 or (new_clusters_pos[j][1] - ori[j][1])>1e-6):
                repeat = True
        print(new_clusters_pos)
        
    for i in range(len(clusters_pts)):
        for pos in (clusters_pts[i]):
            photos[pos[0], pos[1]] = clusters_rgb[i]
    cv2.imwrite('i.jpg', photos)


In [105]:
k_means()

[array([ 94,  99, 100], dtype=uint8), array([84, 77, 68], dtype=uint8), array([115, 113, 119], dtype=uint8)]
[[194.07579155081174, 312.8368463539331], [260.9685118634172, 317.91478674035375], [102.10224984599542, 322.31390270588344]]
[[204.3327996694727, 318.3389626263148], [267.9615087134121, 314.16140961496626], [81.66771618377715, 325.1851504493943]]
[[203.9592764321884, 318.8337349765115], [269.1362739642875, 313.0212833223734], [78.15058302238806, 325.9831389925373]]
[[203.81121225213414, 318.76440266376653], [269.36444399060554, 312.7577606453589], [77.39385162422144, 326.30053100679356]]
[[203.6395176362396, 318.88335459372865], [269.4173404350436, 312.7207597297263], [77.1614221109681, 326.2578196030275]]
[[203.6027510968813, 318.80379165184394], [269.40175207835824, 312.7316204170764], [77.1000602630304, 326.3177634144324]]
[[203.58219330359128, 318.8293586684255], [269.3781551228854, 312.7741972550271], [77.08991088102499, 326.26129352528187]]
[[203.57282589365107, 318.820558

True

In [128]:
def k_means_plus():
    initial_clusters_pos = []
    x, y = random.randrange(photos.shape[0]), random.randrange(photos.shape[1])
    initial_clusters_pos.append([x,y])
    
    distance = []
    position = []
    for i in range(photos.shape[0]):
        for j in range(photos.shape[1]):
            d = np.linalg.norm((photos[i,j] - photos[x,y]), ord=2)
            distance.append(d)
            position.append([i,j])
    unique = np.unique(distance)
    second_d = np.sort(unique)[-2]
    
    idx1 = np.argsort(distance)[-1]
    idx2 = distance.index(second_d)
    
    #print(pos[idx1],pos[idx2])
    
    initial_clusters_pos = [[x,y], position[idx1], position[idx2]]
    print(initial_clusters_pos)
    
    repeat = True

    # mean 顏色
    clusters = []
    for pos in initial_clusters_pos:
        clusters.append(photos[pos[0], pos[1]])
    print(clusters)

    # 同一群pts
    clusters_pts = cal_k_mean(clusters, initial_clusters_pos)

    # 新的mean 位置和顏色
    new_clusters_pos, clusters_rgb = cal_mean(clusters_pts)

    while(repeat):
        repeat = False
        ori = copy.deepcopy(new_clusters_pos)
        clusters_pts = cal_k_mean(clusters_rgb, new_clusters_pos)
        new_clusters_pos, clusters_rgb = cal_mean(clusters_pts)
        for j in range(len(new_clusters_pos)):
            if ((new_clusters_pos[j][0] - ori[j][0])>1e-6 or (new_clusters_pos[j][1] - ori[j][1])>1e-6):
                repeat = True
        print(new_clusters_pos)
    
    for i in range(len(clusters_pts)):
        for pos in (clusters_pts[i]):
            photos[pos[0], pos[1]] = clusters_rgb[i]

    cv2.imwrite('2.jpg', photos)

In [129]:
k_means_plus()

[[28, 337], [190, 0], [0, 4]]
[array([178, 191, 207], dtype=uint8), array([106, 114, 130], dtype=uint8), array([54, 47, 37], dtype=uint8)]
[[77.06397417859803, 326.2579420910134], [203.57379089615932, 318.8546408250356], [269.3679896847991, 312.75886941313144]]
